## Задание 1.

Обучите нейронную сеть решать шифр Цезаря

Что необходимо сделать:.

1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2. Сделать нейронную сеть
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4. Проверить качество

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from time import time
import math
import string
import random
from tqdm.notebook import tqdm
from pathlib import Path
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Параметры обучения
K = 10
SEED = 42
random.seed(SEED)
MAX_LEN = 50

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device=='cpu':
  torch.manual_seed(SEED)
else:
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

device

device(type='cuda')

In [4]:
all_letters='абвгдежзийклмнопрстуфхцчшщъыьэюя '
CHARS = list(all_letters)
INDEX_TO_CHAR =[w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}
n_letters = len(all_letters)
n_letters

33

In [5]:
def caesar_encrypt(text, k):
    """функция для шифрования текста с помощью шифра Цезаря.
    Функция принимает на вход текст и параметр К (сдвиг), и возвращает зашифрованный текст"""
    cipher_text = ''
    for letter in text.lower():
        if letter in CHARS:
            index = (CHARS.index(letter) + k) % len(CHARS)
            cipher_text += CHARS[index]
        else:
            cipher_text += letter
    return cipher_text

In [6]:
original_text = "Зашифрованный текст"
ciphered_text = caesar_encrypt(original_text, K)
print("Зашифрованный текст:", ciphered_text)

Зашифрованный текст: скбтюъшмкччдуйьпфыь


In [7]:
def caesar_decrypt(ciphertext, shift):
    """
    Функция  дешифровки - справочно.
    """
    plaintext = ""
    for char in ciphertext:
      i = INDEX_TO_CHAR.index(char)
      j = i - shift
      if j < 0:
        j +=33
      decrypted_char = INDEX_TO_CHAR[j]
      plaintext += decrypted_char
    return plaintext

In [8]:
decrypted_text = caesar_decrypt(ciphered_text, K)
print("Зашифрованный текст:", decrypted_text)

Зашифрованный текст: зашифрованный текст


In [9]:
with open(Path('data/Kamenev_Maks-Volf_2_Naemnik_RuLit_Me.txt'), 'r') as f1:
    all_text = f1.read().lower()
all_text[:100]

'\n   алекс каменев (alex kamenev)\n   макс вольф 2.\n   наемник.\n   глава 1.\n   станция технического об'

In [10]:
plain_sentences = all_text.split('\n')
# Удаляем все символы, кроме кириллицы и пробелов
pattern = re.compile('[^а-я\s]+')
for i in range(len(plain_sentences)):
    plain_sentences[i] = re.sub(pattern, '', plain_sentences[i])
    plain_sentences[i].strip()
# Заеменяем несколько подряд стоящих пробелов на 1
pattern = re.compile('[\s\s]+')
for i in range(len(plain_sentences)):
    plain_sentences[i] = re.sub(pattern, ' ', plain_sentences[i])
    plain_sentences[i].strip()

plain_sentences = [x[:MAX_LEN] for x in plain_sentences if x != '']

for i in range(len(plain_sentences)):
    plain_sentences[i].strip()

plain_sentences[5:20]

[' то есть как заблокировали стыковочные захваты о ч',
 ' пять минут назад он вызвал меня по внутренней свя',
 ' видимо ролло не слишком мне доверял и установил о',
 ' подобное поведение раздражало хотя и вполне объяс',
 ' бандит боящийся угона своего корабля это что шутк',
 ' нет он действительно любил некроникс и похоже пос',
 ' не думал ты что не мог проконтролировать исходящи',
 ' знаешь что в содружестве делают с пиратами подхож',
 ' знаю не менее яростный ответ показывает что пилот',
 ' недовольно дернувшись логан сатон отсоединил линк',
 ' что можно сделать принудительная отстыковка',
 ' нет отрицательно качание головы в ответ корабль в',
 ' фиксирую движение у главного перехода с внешней с',
 ' а вот и ожил древний искин предтеч и как всегда с',
 ' капитан очевидность блин']

In [11]:
train_plain_sentences, test_plain_sentences = train_test_split(plain_sentences, test_size=0.2, random_state=random.seed(SEED))

In [12]:
def make_cypher(plain_sentences, K):
  
  cypher_sentences = []
  
  for item in plain_sentences:
    cypher_sentences.append(caesar_encrypt(item, K))
  return cypher_sentences

In [13]:
train_cypher_sentences = make_cypher(train_plain_sentences, K)
test_cypher_sentences = make_cypher(test_plain_sentences, K)

In [14]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) 
        return self.out(x)

In [15]:
def pad_sequences(sequences, max_length):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) < max_length:
            padded_sequence = sequence + [0] * (max_length - len(sequence))
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    return padded_sequences

In [16]:
X = torch.Tensor(pad_sequences([[CHAR_TO_INDEX[symb] for symb in line] for line in train_cypher_sentences], MAX_LEN)).long().to(device)
Y = torch.Tensor(pad_sequences([[CHAR_TO_INDEX[symb] for symb in line] for line in train_plain_sentences], MAX_LEN)).long().to(device)
X.shape, Y.shape

(torch.Size([3000, 50]), torch.Size([3000, 50]))

In [17]:
X_test = torch.Tensor(pad_sequences([[CHAR_TO_INDEX[symb] for symb in line] for line in test_cypher_sentences], MAX_LEN)).long().to(device)
Y_test = torch.Tensor(pad_sequences([[CHAR_TO_INDEX[symb] for symb in line] for line in test_plain_sentences], MAX_LEN)).long().to(device)
X.shape, Y.shape

(torch.Size([3000, 50]), torch.Size([3000, 50]))

In [18]:
def evaluate_model(model, data, criterion):
    total_loss = 0.0
    with torch.no_grad():
        for sentences, targets in data:
            output = model(sentences)
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            total_loss += loss.item()
    return total_loss / len(data)

In [19]:
def train_model(model, data, criterion, optimizer, num_epochs):
    for epoch in range(1, num_epochs+1):
        model.train()
        running_loss = 0.0
        for sentences, targets in data:
            optimizer.zero_grad()
            output = model(sentences.to(device))
            loss = criterion(output.view(-1, output.size(-1)), targets.view(-1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        if (epoch) % 10 == 0 or epoch == 1:
            model.eval()
            test_loss = evaluate_model(model, test_data, criterion)
            print(f'Epoch [{epoch}/{num_epochs}], train_loss: {running_loss/len(data)}, test_loss:{test_loss}')

In [20]:
batch_size = 256
data = [(X[i:i+batch_size], Y[i:i+batch_size]) for i in range(0, len(X), batch_size)]
test_data = [(X_test[i:i+batch_size], Y_test[i:i+batch_size]) for i in range(0, len(X_test), batch_size)]

In [21]:
model = RNN(MAX_LEN, 64, MAX_LEN).to(device)

# Определение критерия и оптимизатора
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, data, criterion, optimizer, 100)

Epoch [1/100], train_loss: 3.613961478074392, test_loss:3.129843235015869
Epoch [10/100], train_loss: 0.20259022091825804, test_loss:0.1792769432067871
Epoch [20/100], train_loss: 0.06362259605278571, test_loss:0.06278398384650548
Epoch [30/100], train_loss: 0.045376332476735115, test_loss:0.04604950671394666
Epoch [40/100], train_loss: 0.03932228498160839, test_loss:0.04025315120816231
Epoch [50/100], train_loss: 0.03641301734993855, test_loss:0.037379718075195946
Epoch [60/100], train_loss: 0.034736888172725834, test_loss:0.03567769502600034
Epoch [70/100], train_loss: 0.03366351996858915, test_loss:0.03457572807868322
Epoch [80/100], train_loss: 0.03292670954639713, test_loss:0.03383045891920725
Epoch [90/100], train_loss: 0.0323892078983287, test_loss:0.03330995887517929
Epoch [100/100], train_loss: 0.03197170173128446, test_loss:0.03293973455826441


In [22]:
def generate_text(sentence):
    all_indices = []
    model.eval()

    for line in sentence:
        for s in line:
            s = torch.Tensor(pad_sequences([[CHAR_TO_INDEX[symb] for symb in line] for line in s], MAX_LEN)).long().to(device)
    
            # Передаем в модель тензор категорий, тензор для текущей буквы в шифровке и скрытое состояние
            output = model(s)
    
            # Получаем индекс наиболее вероятной буквы при расшифровке с использованием данного сдвига (категории)
            probas, indices = output.topk(1)
    
            # Собираем все индексы  -  по каждой букве - в один список
            all_indices.append(indices.flatten().cpu().numpy()[0])
            
    # преобразуем индексы букв в сами буквы и получаем расшифровку
    result = ''.join(INDEX_TO_CHAR[i] for i in all_indices)
    return result

In [23]:
corpus = ['Привет ', 'мир ', 'Это ', 'мой ', 'первый ', 'код']
ciphered_text = [caesar_encrypt(original_text, K) for original_text in corpus]

ciphered_text

['щътмпьй', 'цтъй', 'жьшй', 'цшуй', 'щпъмдуй', 'фшо']

In [24]:
generate_text(ciphered_text)

'привет мир это мой первый код'

## Задание 2.
Выполнить практическую работу из лекционного ноутбука.

1. Построить RNN-ячейку на основе полносвязных слоев.
2. Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”.

In [25]:
import numpy as np
import pandas as pd
from collections import Counter
import nltk

In [26]:
def doc2words(doc):
  # мешок слов
    words=[]
    for line in doc:
      words += line.split()
    return words

In [27]:
def removepunct(words):
  # Удаляем пунктуацию
    punct = set(string.punctuation)
    words = [''.join([char for char in list(word) if char not in punct]) for word in words]
    return words

In [28]:
def getvocab(words):
  # Словарь из мешка слов
    wordfreq = Counter(words)
    sorted_wordfreq = sorted(wordfreq, key=wordfreq.get)
    return sorted_wordfreq

In [29]:
def vocab_map(vocab):
    # 2 словаря - int to words and word to int
    int_to_vocab = {k:w for k,w in enumerate(vocab)}
    vocab_to_int = {w:k for k,w in int_to_vocab.items()}
    return int_to_vocab, vocab_to_int

In [30]:
def get_batches(words, vocab_to_int, batch_size, seq_size):
    # Генерируем батчи для  Xs и Ys: shape = (batchsize * num_batches) * seq_size
    word_ints = [vocab_to_int[word] for word in words]
    num_batches = int(len(word_ints) / (batch_size * seq_size))
    Xs = word_ints[:num_batches*batch_size*seq_size]
    Ys = np.zeros_like(Xs)
    Ys[:-1] = Xs[1:]
    Ys[-1] = Xs[0]
    Xs = np.reshape(Xs, (num_batches*batch_size, seq_size))
    Ys = np.reshape(Ys, (num_batches*batch_size, seq_size))
    
    for i in range(0, num_batches*batch_size, batch_size):
        yield Xs[i:i+batch_size, :], Ys[i:i+batch_size, :]

In [31]:
class RNNModule(nn.Module):

    def __init__(self, n_vocab, seq_size=32, embedding_size=64, lstm_size=32):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),torch.zeros(1, batch_size, self.lstm_size))

In [32]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer

In [33]:
def generate_txt(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [34]:
def train_rnn(words, vocab_to_int, int_to_vocab, n_vocab):
    
    # ячейка RNN
    net = RNNModule(n_vocab, 32, 64, 64)
    net = net.to(device)
    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0
    
    # итерируемся по эпохам
    for e in tqdm(range(5)):
        # получаем батчи
        batches = get_batches(words, vocab_to_int, 32, 32)
        # инициализируем выход и сккрытое состояние
        state_h, state_c = net.zero_state(32)

        # Передаем данные на GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        # итерируемся по батчам
        for x, y in tqdm(batches):
            iteration += 1

            # Переходим  в режим обучения
            net.train()

            # Обнуляем градиенты
            optimizer.zero_grad()

            # Передаем x и y на GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)
            
            # Модель возвращает логиты, последнее скрытое состояние и новый выход
            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y.long())

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # back-propagation
            loss.backward(retain_graph=True)

            _ = torch.nn.utils.clip_grad_norm_(net.parameters(), 5)
            
            # Обновляем параметры, выполняя шаг обучения
            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 200),'Iteration: {}'.format(iteration),'Loss: {}'.format(loss_value))

            # if iteration % 1000 == 0:
                # predict(device, net, flags.initial_words, n_vocab,vocab_to_int, int_to_vocab, top_k=5)
                # torch.save(net.state_dict(),'checkpoint_pt/model-{}.pth'.format(iteration))
                
    return net

In [35]:
doc = pd.read_csv('data/simpsons_script_lines.csv', usecols=['normalized_text'],low_memory=False)['normalized_text'].astype(str).to_list()
doc[:5]

['no actually it was a little of both sometimes when a disease is in all the magazines and all the news shows its only natural that you think you have it',
 'wheres mr bergstrom',
 'i dont know although id sure like to talk to him he didnt touch my lesson plan what did he teach you',
 'that life is worth living',
 'the polls will be open from now until the end of recess now just in case any of you have decided to put any thought into this well have our final statements martin']

In [36]:
# предобработка
words = removepunct(doc2words(doc))
vocab = getvocab(words)
int_to_vocab, vocab_to_int = vocab_map(vocab)

In [37]:
rnn_net = train_rnn(words, vocab_to_int, int_to_vocab, len(vocab))

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epoch: 0/200 Iteration: 100 Loss: 6.783677101135254
Epoch: 0/200 Iteration: 200 Loss: 6.454249382019043
Epoch: 0/200 Iteration: 300 Loss: 6.178114891052246
Epoch: 0/200 Iteration: 400 Loss: 6.626694202423096
Epoch: 0/200 Iteration: 500 Loss: 6.342596054077148
Epoch: 0/200 Iteration: 600 Loss: 6.122922897338867
Epoch: 0/200 Iteration: 700 Loss: 6.010077953338623
Epoch: 0/200 Iteration: 800 Loss: 6.387012004852295
Epoch: 0/200 Iteration: 900 Loss: 5.8750481605529785
Epoch: 0/200 Iteration: 1000 Loss: 6.584317684173584
Epoch: 0/200 Iteration: 1100 Loss: 6.211277484893799
Epoch: 0/200 Iteration: 1200 Loss: 6.405735969543457
Epoch: 0/200 Iteration: 1300 Loss: 6.00385046005249


0it [00:00, ?it/s]

Epoch: 1/200 Iteration: 1400 Loss: 5.719114780426025
Epoch: 1/200 Iteration: 1500 Loss: 5.99172830581665
Epoch: 1/200 Iteration: 1600 Loss: 5.833299160003662
Epoch: 1/200 Iteration: 1700 Loss: 5.675997734069824
Epoch: 1/200 Iteration: 1800 Loss: 5.862273216247559
Epoch: 1/200 Iteration: 1900 Loss: 5.9734272956848145
Epoch: 1/200 Iteration: 2000 Loss: 5.61801815032959
Epoch: 1/200 Iteration: 2100 Loss: 5.313600063323975
Epoch: 1/200 Iteration: 2200 Loss: 5.647763729095459
Epoch: 1/200 Iteration: 2300 Loss: 5.71717643737793
Epoch: 1/200 Iteration: 2400 Loss: 5.839413166046143
Epoch: 1/200 Iteration: 2500 Loss: 5.418987274169922
Epoch: 1/200 Iteration: 2600 Loss: 5.562053203582764


...

Epoch: 2/200 Iteration: 2700 Loss: 5.683876991271973
Epoch: 2/200 Iteration: 2800 Loss: 5.37455415725708
Epoch: 2/200 Iteration: 2900 Loss: 5.4336442947387695
Epoch: 2/200 Iteration: 3000 Loss: 5.437780857086182
Epoch: 2/200 Iteration: 3100 Loss: 5.489935398101807
Epoch: 2/200 Iteration: 3200 Loss: 5.647677898406982
Epoch: 2/200 Iteration: 3300 Loss: 5.664626598358154
Epoch: 2/200 Iteration: 3400 Loss: 5.139298915863037
Epoch: 2/200 Iteration: 3500 Loss: 5.526797771453857
Epoch: 2/200 Iteration: 3600 Loss: 5.244123935699463
Epoch: 2/200 Iteration: 3700 Loss: 5.480949878692627
Epoch: 2/200 Iteration: 3800 Loss: 5.358027935028076
Epoch: 2/200 Iteration: 3900 Loss: 5.541860103607178


0it [00:00, ?it/s]

Epoch: 3/200 Iteration: 4000 Loss: 5.370248317718506
Epoch: 3/200 Iteration: 4100 Loss: 5.236973762512207
Epoch: 3/200 Iteration: 4200 Loss: 5.312046527862549
Epoch: 3/200 Iteration: 4300 Loss: 5.391300201416016
Epoch: 3/200 Iteration: 4400 Loss: 5.438663482666016
Epoch: 3/200 Iteration: 4500 Loss: 5.197635173797607
Epoch: 3/200 Iteration: 4600 Loss: 5.533201694488525
Epoch: 3/200 Iteration: 4700 Loss: 5.294881820678711
Epoch: 3/200 Iteration: 4800 Loss: 5.259171962738037
Epoch: 3/200 Iteration: 4900 Loss: 5.44784688949585
Epoch: 3/200 Iteration: 5000 Loss: 5.521522045135498
Epoch: 3/200 Iteration: 5100 Loss: 5.545651912689209
Epoch: 3/200 Iteration: 5200 Loss: 5.349438667297363


0it [00:00, ?it/s]

Epoch: 4/200 Iteration: 5300 Loss: 5.264430999755859
Epoch: 4/200 Iteration: 5400 Loss: 5.356697082519531
Epoch: 4/200 Iteration: 5500 Loss: 5.2102251052856445
Epoch: 4/200 Iteration: 5600 Loss: 5.366838455200195
Epoch: 4/200 Iteration: 5700 Loss: 5.170050621032715
Epoch: 4/200 Iteration: 5800 Loss: 5.330789089202881
Epoch: 4/200 Iteration: 5900 Loss: 5.033596515655518
Epoch: 4/200 Iteration: 6000 Loss: 5.226826190948486
Epoch: 4/200 Iteration: 6100 Loss: 5.091064929962158
Epoch: 4/200 Iteration: 6200 Loss: 5.061532497406006
Epoch: 4/200 Iteration: 6300 Loss: 5.26991605758667
Epoch: 4/200 Iteration: 6400 Loss: 5.2263994216918945
Epoch: 4/200 Iteration: 6500 Loss: 5.263146877288818


In [40]:
torch.save(rnn_net.state_dict(), 'rnn')

In [42]:
generate_txt(device, rnn_net, ['hey', 'you'], len(vocab), vocab_to_int, int_to_vocab)

hey you know i want you i want the money nan i cant get the money to you i think that were you gonna get to you nan oh my little league nan hey homer i think youre right i know you want me right to me you dont know i dont want to get a few boners i want a big kid you have you to tell you about it nan hello i dont know i know you have a big man i want a big kid who is the greatest day to make you have the way you ever want you
